In [1]:
import numpy as np
import json
import os
import tensorflow as tf
import onnx

In [2]:
import os
import json
import numpy as np
import tensorflow as tf

# Save tensor as binary file
def save_tensor_as_bin(tensor, filepath):
    np_array = tensor.numpy() if isinstance(tensor, tf.Tensor) else tensor
    np_array.tofile(filepath)

# Save JSON file
def save_json(data, filepath):
    with open(filepath, 'w') as f:
        json.dump(data, f, indent=4)

# Create directories
def create_dir(dir_path):
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

In [3]:
# Layer-wise dump with a single JSON for the model architecture
def layer_wise_dump_tf(model, input_tensor, save_dir):
    # Ensure save directory structure
    create_dir(save_dir)
    create_dir(f"{save_dir}/input")
    create_dir(f"{save_dir}/output")
    create_dir(f"{save_dir}/weights")
    create_dir(f"{save_dir}/reference")

    model_architecture = {
        "model_name": model.name,
        "num_layers": len(model.layers),
        "input_shape": list(model.input_shape) if hasattr(model, 'input_shape') else "Unknown",
        "output_shape": list(model.output_shape) if hasattr(model, 'output_shape') else "Unknown",
        "layers": []
    }

    x = input_tensor
    for layer in model.layers:
        layer_name = layer.name

        # Save input tensor
        input_filepath = f"{save_dir}/input/{layer_name}_input.bin"
        save_tensor_as_bin(x, input_filepath)

        # Save weights
        weight_filepaths = []
        if hasattr(layer, 'weights') and layer.weights:
            for weight in layer.weights:
                weight_name = weight.name.split("/")[-1].replace(":", "_")
                weight_filepath = f"{save_dir}/weights/{layer_name}_{weight_name}.bin"
                save_tensor_as_bin(weight.numpy(), weight_filepath)
                weight_filepaths.append(weight_filepath)

        # Compute layer output
        x = layer(x)

        # Save output tensor
        output_filepath = f"{save_dir}/output/{layer_name}_output.bin"
        save_tensor_as_bin(x, output_filepath)

        # Gather layer attributes
        layer_attributes = {
            "layer_name": layer_name,
            "type": layer.__class__.__name__,
            "input_shape": list(layer.input_shape) if hasattr(layer, 'input_shape') else "Unknown",
            "output_shape": list(layer.output_shape) if hasattr(layer, 'output_shape') else "Unknown",
            "input_file": input_filepath,
            "output_file": output_filepath,
            "weight_files": weight_filepaths,
        }
        if hasattr(layer, 'kernel_size'):
            layer_attributes["kernel_size"] = layer.kernel_size
        if hasattr(layer, 'strides'):
            layer_attributes["strides"] = layer.strides
        if hasattr(layer, 'padding'):
            layer_attributes["padding"] = layer.padding
        if hasattr(layer, 'activation') and layer.activation:
            layer_attributes["activation"] = layer.activation.__name__

        # Add layer attributes to the model architecture JSON
        model_architecture["layers"].append(layer_attributes)

    # Save the complete model architecture to a single JSON file
    save_json(model_architecture, f"{save_dir}/model_architecture.json")

    return model_architecture

In [4]:
model = tf.keras.models.load_model("C:/Users/HP/Operators_Assignment/cnn_cifar10_model.h5")

In [5]:
input_tensor = tf.random.normal([1, 32, 32, 3])  # Random input for CIFAR-10
layer_wise_dump_tf(model, input_tensor, save_dir="data")

{'model_name': 'sequential',
 'num_layers': 8,
 'input_shape': [None, 32, 32, 3],
 'output_shape': [None, 10],
 'layers': [{'layer_name': 'conv2d',
   'type': 'Conv2D',
   'input_shape': [None, 32, 32, 3],
   'output_shape': [None, 30, 30, 32],
   'input_file': 'data/input/conv2d_input.bin',
   'output_file': 'data/output/conv2d_output.bin',
   'weight_files': ['data/weights/conv2d_kernel_0.bin',
    'data/weights/conv2d_bias_0.bin'],
   'kernel_size': (3, 3),
   'strides': (1, 1),
   'padding': 'valid',
   'activation': 'relu'},
  {'layer_name': 'max_pooling2d',
   'type': 'MaxPooling2D',
   'input_shape': [None, 30, 30, 32],
   'output_shape': [None, 15, 15, 32],
   'input_file': 'data/input/max_pooling2d_input.bin',
   'output_file': 'data/output/max_pooling2d_output.bin',
   'weight_files': [],
   'strides': (2, 2),
   'padding': 'valid'},
  {'layer_name': 'conv2d_1',
   'type': 'Conv2D',
   'input_shape': [None, 15, 15, 32],
   'output_shape': [None, 13, 13, 64],
   'input_file': 

In [6]:
import os
import json
import numpy as np

def generate_layer_config(model, save_dir, config_file_path):
    # Build model or run a dummy forward pass to infer shapes
    if not model.built:
        # Infer the input shape from the model's first layer
        input_shape = model.input_shape if hasattr(model, 'input_shape') else None
        if input_shape:
            model.build(input_shape)
        else:
            # Use dummy input if input_shape is unknown
            dummy_input = np.random.random((1, *model.layers[0].input_shape[1:]))
            model(dummy_input)
    
    layer_configs = []  # List to store layer-wise configurations

    for layer in model.layers:
        layer_name = layer.name

        # Construct layer configuration
        layer_config = {
            "layer_name": layer_name,
            "type": layer.__class__.__name__,  # Correct way to get the class name
            "input_file_path": f"{save_dir}/input/{layer_name}_input.bin",
            "output_file_path": f"{save_dir}/output/{layer_name}_output.bin",
            "weights_file_paths": [],  # To be filled if the layer has weights
            "attributes": {
                "input_shape": list(layer.input_shape) if hasattr(layer, 'input_shape') else "Unknown",
                "output_shape": list(layer.output_shape) if hasattr(layer, 'output_shape') else "Unknown",
            }
        }

        # Add weights and biases if applicable
        if hasattr(layer, 'weights') and layer.weights:
            for weight in layer.weights:
                weight_name = weight.name.split("/")[-1].replace(":", "_")
                weight_file_path = f"{save_dir}/weights/{layer_name}_{weight_name}.bin"
                layer_config["weights_file_paths"].append(weight_file_path)

        # Add other layer-specific attributes if available
        if hasattr(layer, 'kernel_size'):
            layer_config["attributes"]["kernel_size"] = layer.kernel_size
        if hasattr(layer, 'strides'):
            layer_config["attributes"]["strides"] = layer.strides
        if hasattr(layer, 'padding'):
            layer_config["attributes"]["padding"] = layer.padding
        if hasattr(layer, 'activation') and layer.activation:
            layer_config["attributes"]["activation"] = layer.activation.__name__

        # Append this layer's config to the list
        layer_configs.append(layer_config)

    # Save all configurations to a JSON file
    with open(config_file_path, 'w') as config_file:
        json.dump({"layers": layer_configs}, config_file, indent=4)

    print(f"Configuration file saved to {config_file_path}")

In [7]:
# Assuming the `data` folder is in the current directory
data_folder = "C:/Users/HP/Operators_Assignment/data"
output_json_file = "C:/Users/HP/Operators_Assignment/configs/json/model_config.json"
model = tf.keras.models.load_model("C:/Users/HP/Operators_Assignment/cnn_cifar10_model.h5")

generate_layer_config(model, data_folder, output_json_file)

Configuration file saved to C:/Users/HP/Operators_Assignment/configs/json/model_config.json


In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 15, 15, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 6, 6, 64)          0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 4, 4, 128)         73856     
                                                                 
 flatten (Flatten)           (None, 2048)              0

In [10]:
 model.layers[0].input_shape[1:]

(32, 32, 3)

In [2]:
import json


config_file_path = "C:/Users/HP/Operators_Assignment/configs/json/model_config.json"
with open(config_file_path, "r") as json_file:
    config = json.load(json_file)

# Access layers array from config
layers = config["layers"]

In [7]:
for layer in layers:
    print(layer['layer_name'])

conv2d
max_pooling2d
conv2d_1
max_pooling2d_1
conv2d_2
flatten
dense
dense_1
